# Inventory Modeling and Analysis Made Easy with Supplychainpy
** *by Kevin Fasusi* **

This workbook assumes some familiarity and proficiency in programming with Python. Understanding list comprehensions, conditional logic and functions are an essential prerequisite for continuing with this workbook.

Typically, inventory analysis using Excel requires several formulas, manual processes, possibly some pivot tables and in some cases VBA to achieve. Using the supplychainpy library can reduce the time taken and effort made for the same analysis.

In [1]:
from supplychainpy.model_inventory import analyse
from decimal import Decimal
from supplychainpy.sample_data.config import ABS_FILE_PATH

The first two imports are mandatory; the second import is necessary for using the sample data in the `supplychainpy` library. When working with a different file, supply the file path to the `file_path` parameter. The data provided for analysis can be from a `CSV` or a database ETL process.

The sample data is a `CSV` formatted file:

In [2]:
with open(ABS_FILE_PATH['COMPLETE_CSV_SM'],'r') as raw_data:
    for line in raw_data:
        print(line)

Sku,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,unit cost,lead-time,retail_price,quantity_on_hand,backlog

KR202-209,1509,1855,2665,1841,1231,2598,1988,1988,2927,2707,731,2598,1001,2,5000,1003,10

KR202-210,1006,206,2588,670,2768,2809,1475,1537,919,2525,440,2691,394,2,1300,3224,10

KR202-211,1840,2284,850,983,2737,1264,2002,1980,235,1489,218,525,434,4,1200,390,10

KR202-212,104,2262,350,528,2570,1216,1101,2755,2856,2381,1867,2743,474,3,10,390,10

KR202-213,489,954,1112,199,919,330,561,2372,921,1587,1532,1512,514,1,2000,2095,10

KR202-214,2416,2010,2527,1409,1059,890,2837,276,987,2228,1095,1396,554,2,1800,55,10

KR202-215,403,1737,753,1982,2775,380,1561,1230,1262,2249,824,743,594,1,2500,4308,10

KR202-216,2908,929,684,2618,1477,1508,765,43,2550,2157,937,1201,634,3,3033,34,10

KR202-217,2799,2197,1647,2263,224,2987,2366,588,1140,869,1707,1180,674,3,5433,390,10

KR202-218,1333,402,804,318,1408,830,1028,534,1871,2730,2022,94,714,2,3034,3535,10

KR202-219,813,969,745,1001,2732,1987,717,

It is probable that getting the data into this format will require 'extracting' from a database and 'transforming' data before 'loading' into the `analyse` function. To perform the ETL, an ORM like SQLAlchemy or appropriate DB-API for the database in question may be of use. Supplychainpy works with Pandas, so performing the transformations using Pandas may be an idea. The `DataFrame` or file passed as an argument must be identical to the format above (future versions of supplychainpy will be more lenient and attempt to identify if a minimum requirement is satisfied).

The ETL process is not covered in this workbook but on the 'roadmap' for `supplychainpy` is the automation of this process. 

So now that we have the data in the correct format, we can proceed with the analysis.

In [3]:
#%%timeit
analysed_inventory_profile= analyse(file_path=ABS_FILE_PATH['COMPLETE_CSV_SM'],
                                                             z_value=Decimal(1.28),
                                                             reorder_cost=Decimal(400),
                                                             file_type='csv')


The variable `analysed_inventory_profile` now contains a collection (list) of `UncertainDemand` objects (one per SKU). Each object contains the analysis for each SKU. The analysis include the following:

- safety stock
- total_orders
- standard_deviation
- quantity_on_hand': '1003
- economic_order_variable_cost
- sku
- economic_order_quantity
- unit_cost
- demand_variability
- average_orders
- excess_stock
- currency
- ABC_XYZ_Classification
- shortages
- reorder_level
- revenue 
- reorder_quantity 
- safety_stock
- orders

The listed summary items can be retrieved by calling the method `orders_summary()` on each object. The quickest way to do this is with a list comprehension.

In [6]:
analysis_summary = [demand.orders_summary() for demand in analysed_inventory_profile]
print(analysis_summary)

[{'reorder_level': '4069', 'unit_cost': '1001', 'economic_order_variable_cost': '15708.41', 'reorder_quantity': '13', 'safety_stock': '1165', 'quantity_on_hand': '1003', 'shortages': '5969', 'sku': 'KR202-209', 'revenue': '123190000', 'average_orders': '2053.1667', 'standard_deviation': '644', 'currency': 'UNKNOWN', 'economic_order_quantity': '44', 'demand_variability': '0.314', 'total_orders': '24638', 'ABC_XYZ_Classification': 'BY', 'excess_stock': '0', 'orders': {'demand': ('1509', '1855', '2665', '1841', '1231', '2598', '1988', '1988', '2927', '2707', '731', '2598')}}, {'reorder_level': '4033', 'unit_cost': '394', 'economic_order_variable_cost': '8797.61', 'reorder_quantity': '18', 'safety_stock': '1719', 'quantity_on_hand': '3224', 'shortages': '0', 'sku': 'KR202-210', 'revenue': '25524200', 'average_orders': '1636.1667', 'standard_deviation': '950', 'currency': 'UNKNOWN', 'economic_order_quantity': '63', 'demand_variability': '0.580', 'total_orders': '19634', 'ABC_XYZ_Classificat

For the intrepid reader who did not heed the warning about the prerequisites and is now scratching their head wondering "what manner of black magic is this," here is a quick overview on list comprehensions. In short, the above code is similar to the code below:

In [5]:
analysis_summary =[]
for demand in analysed_inventory_profile:
    analysis_summary.append(demand.orders_summary())

The former is much more readable.

## Exploring the results

To make sense of the results, we can filter our analysis using standard Python scripting techniques. For example to retrieve the whole `summary` for the SKU `KR202-209` we can do something like this:

In [7]:
%%timeit
sku_summary = [demand.orders_summary() for demand in analysed_inventory_profile if demand.orders_summary().get('sku')== 'KR202-209']


1000 loops, best of 3: 979 µs per loop


The inventory classification ABC XYZ denotes the SKUs contribution to revenue and demand volatility. `AX` SKUs typically exhibit steady demand and contribute 80% of the value of income for the period under analysis.

As a more traditional way of grouping SKUs by behaviour, it is also likely to be used for generating inventory policies and for further exploration of the stock profile. To retrieve all the summaries for a particular classification, we could do something like this:

In [7]:
ay_classification_summary = [demand.orders_summary() for demand in analysed_inventory_profile if demand.orders_summary().get('ABC_XYZ_Classification')== 'AY']
print(ay_classification_summary)

[{'economic_order_quantity': '45', 'ABC_XYZ_Classification': 'AY', 'sku': 'KR202-225', 'shortages': '10542', 'demand_variability': '0.344', 'reorder_level': '7402', 'reorder_quantity': '13', 'unit_cost': '994', 'currency': 'UNKNOWN', 'standard_deviation': '721', 'revenue': '226639815', 'average_orders': '2098.75', 'safety_stock': '2261', 'quantity_on_hand': '2000', 'orders': {'demand': ('2744', '2770', '2697', '1726', '1776', '2264', '332', '2420', '2722', '1161', '1986', '2587')}, 'excess_stock': '0', 'economic_order_variable_cost': '15826.20', 'total_orders': '25185'}, {'economic_order_quantity': '32', 'ABC_XYZ_Classification': 'AY', 'sku': 'KR202-229', 'shortages': '0', 'demand_variability': '0.594', 'reorder_level': '3153', 'reorder_quantity': '9', 'unit_cost': '1154', 'currency': 'UNKNOWN', 'standard_deviation': '752', 'revenue': '197613000', 'average_orders': '1266.75', 'safety_stock': '1362', 'quantity_on_hand': '8939', 'orders': {'demand': ('2114', '198', '1479', '1249', '1475'

Using a built-in feature of the library provides a quicker way to filter the results. For example a quciker way to filter for SKU `KR202-209`, is through the use of `Inventory` class in the `summarise` module.

In [8]:
from supplychainpy.inventory.summarise import Inventory
filtered_summary = Inventory(analysed_inventory_profile)

In [9]:
%%timeit
sku_summary = [summary for summary in filtered_summary.describe_sku('KR202-209')]
#print(sku_summary)

1000 loops, best of 3: 188 µs per loop


Using the import Inventory specifically built to filter the analysis is faster and syntactically cleaner for easier to read and understand the code. The `Inventory` summary class also provides a more detailed overview of the SKU with additional KPIs and metric in the context of the whole stock profile. The `orders_summary` method ranks and performs some comparative analysis for more insight.

The descriptive summary includes:

- shortage_rank
- min_orders
- excess_units
- revenue_rank
- excess_rank
- average_orders
- gross_profit_margin
- markup_percentage
- max_order
- shortage_cost
- quantity_on_hand
- inventory_turns
- sku_id
- retail_price
- revenue_rank 
- shortage_units
- unit_cost
- classification
- safety_stock_cost
- safety_stock_units
- safety_stock_rank
- percentage_contribution_revenue
- gross_profit_margin
- shortage_rank
- inventory_traffic_light
- unit_cost_rank
- excess_cost
- excess_units
- markup_percentage
- revenue

Further summaries can be retrieved, for instance, reviews at the inventory classification level of detail can be quite useful when exploring inventory policies:

In [10]:
classification_summary =  [summary for summary in filtered_summary.abc_xyz_summary(classification=('AY',), category=('revenue',))]
print(classification_summary)

[{'AY': {'revenue': 5372496600.0}}]


Now we know the total revenue generated by the `AY` SKU class. There is another, slightly more fun way to arrive at this number using `Dash` but more on that later.

In [11]:
top_10_safety_stock_skus =  [summary.get('sku')for summary in filtered_summary.rank_summary(attribute='safety_stock', count=10)]
print(top_10_safety_stock_skus)

['KR202-241', 'KR202-231', 'KR202-233', 'KR202-227', 'KR202-225', 'KR202-212', 'KR202-240', 'KR202-244', 'KR202-236', 'KR202-211', 'KR202-243']


Let us add the `safety_stock` and create a tuple to see that the results explicitly.

In [12]:
top_10_safety_stock_values =  [(summary.get('sku'), summary.get('safety_stock'))for summary in filtered_summary.rank_summary(attribute='safety_stock', count=10)]
print(top_10_safety_stock_values)

[('KR202-241', '2719'), ('KR202-231', '2484'), ('KR202-233', '2472'), ('KR202-227', '2277'), ('KR202-225', '2261'), ('KR202-212', '2164'), ('KR202-240', '2120'), ('KR202-244', '2054'), ('KR202-236', '2045'), ('KR202-211', '2020'), ('KR202-243', '1954')]


We can then pass back the list of `top_10_safety_stock_skus` back into the inventory filter and get their breakdown.

In [13]:
top_10_safety_stock_summary = [summary for summary in filtered_summary.describe_sku(*top_10_safety_stock_skus)]
#print(top_10_safety_stock_summary)

We have only covered a few use cases, but we have already achieved a significant amount of analysis with relatively few line of code. The equivalent in Excel would require much more work and many more formulas.